# Interactive neural network display

## Setting up
#### 1. Importing Dependency


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


import random

import pandas as pd
import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from random import randrange



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Structuring the networks
#### 6. The 1st layer of the hyperspectral network


In [2]:
def hyperspectral(input_,n_o_input, keep_prob, filter_width = 1, stride_size =1, relu_alpha = 0.2):
    n_o_strides = int((n_o_input-filter_width)/stride_size) +1  #round down
   
    Hyper_layer = []
    
    def dense(input_,start,keep_prob, filter_width, stride_size, relu_alpha):
        nn_input = tf.slice(input_,[0,start],[-1,filter_width])
        
        dropout1 = tf.nn.dropout(nn_input, keep_prob)
        dense1 = tf.layers.dense(dropout1, 1)
        relu1 = tf.maximum(relu_alpha * dense1, dense1)        
        return relu1
    
    for step in range(n_o_strides):
        start = step*stride_size
        output = dense(input_,start,keep_prob, filter_width, stride_size, relu_alpha)
        Hyper_layer.append(output)
    
    if (n_o_input-filter_width)%stride_size>0:
        start = n_o_input-filter_width
        output = dense(input_,start,keep_prob, filter_width, stride_size, relu_alpha)
        Hyper_layer.append(output)
        
    Hyper_l_stacked = tf.concat(Hyper_layer,1)
    
    print("Hyper_l_stacked",Hyper_l_stacked)
    return Hyper_l_stacked , n_o_strides

#### 7. The remaining neural network layers

In [3]:
def Classifier(input_,n_o_class,n_o_input, keep_prob,relu_alpha = 0.2):
    print("n_o_input",n_o_input)
    if n_o_input == 3:
        is_RGB = True
    elif n_o_input == 571:
        is_RGB = False
    else:
        raise ValueError('A very specific bad thing happened.'+str(n_o_input))
    
    if is_RGB:
        dense0 = tf.layers.dense(input_, 3)    
        relu0 = tf.maximum(relu_alpha * dense0, dense0)
        first_layer_out = tf.nn.dropout(relu0, keep_prob)
    else:
        print(input_,n_o_input, keep_prob)
        first_layer_out,n_o_input= hyperspectral(input_,n_o_input, keep_prob, filter_width = 30, stride_size =1, relu_alpha = 0.2)

    hidden_size = n_o_input*2/3
    hidden_nodes = int(hidden_size)+1 # rounding
    print("hidden size:",str(hidden_nodes))
    
    
    dense1 = tf.layers.dense(first_layer_out, hidden_nodes)    
    relu1 = tf.maximum(relu_alpha * dense1, dense1)
    dropout1 = tf.nn.dropout(relu1, keep_prob)
    
    
    class_logits = tf.layers.dense(dropout1, n_o_class)    
    
    return class_logits

#### 10. Function to define loss value for training
Inspired by the aggregate loss scoring system from a GAN semi-supervised networ, an aggregated scoring system was structured to calculate the network’s loss value for training. This averaged the loss between the main class classification and the subclass classification; the main classes being manure vs non-manure, and the subclasses being the specific animal class or material type. And this aggregated scoring system aimed to teach the network the relationship between the groups of subclasses. 


In [4]:
def softmax(input_,m_class, n_class,n_o_input, keep_prob,relu_alpha = 0.2,sub_scaling = 1):
    
    n_o_class = m_class+n_class
    
    #raw output
    logits= Classifier(input_,n_o_class,n_o_input, keep_prob,relu_alpha = 0.2)
    subclass_softmax = tf.nn.softmax(logits)
    
    #Reduce outputs from 8 subclasses to 2 main classes
    m_class_logit, n_class_logit = tf.split(logits, [m_class, n_class], 1)
    m_class_logit1 =tf.reduce_sum(m_class_logit,1, keepdims =True) 
    n_class_logit1 =tf.reduce_sum(n_class_logit,1, keepdims =True) 
    main_class_logits = tf.concat([m_class_logit1, n_class_logit1], 1)
    main_class_softmax = tf.nn.softmax(main_class_logits)

    return subclass_softmax,main_class_softmax

## Training the Model
#### 11. Defining the training parameters

In [5]:
tf.reset_default_graph()



In [7]:

import random
import pandas as pd
import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from random import randrange
from ipywidgets import interactive, interact , FloatSlider,widgets,VBox,HBox,interactive_output,Layout,Output
from IPython.display import display,HTML,clear_output
from scipy.interpolate import UnivariateSpline

RGB = []
hyperspectral = []

box1 = Output(layout={'width': '25%','border': '1px solid black'})
box2 = Output(layout={'width': '35%','border': '1px solid black' })
box3 = Output(layout={'width': '30%', 'border': '1px solid black'})
box4 = Output(layout={'width': '45%','border': '1px solid black' })
box5 = Output(layout={'width': '45%','border': '1px solid black' })

slider1 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='380nm',layout= Layout(width='99%'))
slider2 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='450nm',layout= Layout(width='99%'))
slider3 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='550nm',layout= Layout(width='99%'))
slider4 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='650nm',layout= Layout(width='99%'))
slider5 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='750nm',layout= Layout(width='99%'))
slider6 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='850nm',layout= Layout(width='99%'))
slider7 = FloatSlider(min=0, max=1, step=0.01, value=0.5,description='950nm',layout= Layout(width='99%'))

def click(event):
    classify(RGB,hyperspectral)
    
button = widgets.Button(
    description='Run Classification',
    layout={'width': '130px','margin':'0px 0px 0px 15%'}
)
button.on_click(click)


slider_list = [slider1,slider2,slider3,slider4,slider5,slider6,slider7,button]




for slider in slider_list:
    with box3:
        display(slider)

def classify(RGB,hyperspectral):
    keep_probability = 1
    def print_network_output(save_model_path,reflectance_data,n_o_input):
        tf.reset_default_graph()
        n_class = 3
        m_class = 5
        n_o_class = m_class+n_class
        epochs = 10
        keep_probability = 0.95
        sub_scaling = 1 
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
        input_ = tf.placeholder(tf.float32,  [None,n_o_input],name = 'x')
        subclass_softmax,main_class_softmax =softmax(input_,m_class, n_class,n_o_input, keep_prob,relu_alpha = 0.2,sub_scaling = sub_scaling) 
        clear_output(wait=True)
        print('Loading...')
        print_network_output(save_model_path,[hyperspectral],571)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            loader = tf.train.import_meta_graph(save_model_path + '.meta')
            loader.restore(sess, save_model_path)
            subclass_softmax_p,main_class_softmax_p= sess.run([subclass_softmax,main_class_softmax], feed_dict = {input_:reflectance_data,keep_prob:keep_probability})

            clear_output(wait=True)
            print("Network Classification")
            print("Main Class")
            main_class_softmax_p  = main_class_softmax_p[0]
            print(" - "+str(main_class_softmax_p[0]))
            print(" - "+str(main_class_softmax_p[1]))
            print("Subclass")
            subclass_softmax_p = subclass_softmax_p[0]
            for subclass in subclass_softmax_p:
                print(" - " +str(subclass))
    print_network_output('../training_Hyperspectral',[hyperspectral],571)

"""    with box4:
        save_model_path = '../training_Hyperspectral'
        print_network_output(save_model_path,[hyperspectral],571)
    with box5:
        save_model_path = '../training_rgb'
        print_network_output(save_model_path,[RGB],3)"""



            
def f(z,a,b,c,d,e,f,g):
    if z == "Select":
        for slider in slider_list:
            slider.layout.display = 'none'
        return 0
    data_path ="../Training data/averaged all.csv"
    data = pd.read_csv(data_path)
    data.head()
    target_fields ='Class'
    data = data.drop(["Class code"],axis=1)
    features0, targets0 = data.drop(target_fields, axis=1), data[target_fields]
    features, targets  = np.array(features0) , np.array(targets0)

    dic = {}
    for feature,target in zip(features, targets):
        dic[target]=list(feature)
    if z == "Make your own":
        x= [380,450,550,650,750,850,950]
        y = [a,b,c,d,e,f,g]
        plt.scatter(x, y)
        plt.plot(x, y, 'o')
    else:
        x= [x for x in range(380,951)]
        y = dic[z.replace("Average ","")]     
        for slider in slider_list:
            slider.layout.display = 'none'
            
    graph = UnivariateSpline(x, y, s=0)
    xs = np.linspace(380, 950, 100)
    ys = graph(xs)
    
    with box2:
        clear_output(wait=True)

        plt.ylim(0,1)
        plt.xlim(380, 950)
        plt.plot(xs, ys)
        plt.title("Reflectance spectrum")
        plt.show()
        
    global hyperspectral
    global RGB
    hyperspectral = [ min(max(round(float(graph(x)),7),0),1) for x in range(380,951)]
    red =  min(max(round(float(graph(680)),4),0),1)
    green =  min(max(round(float(graph(530)),4),0),1)
    blue =  min(max(round(float(graph(465)),4),0),1)
    RGB = [red,green,blue]
    
    with box1:
        clear_output(wait=True)
        red_rgb = int(red*255)
        green_rgb = int(green*255)
        blue_rgb = int(blue*255)
        #TODO: align central 
        html = '<svg height="140" width="100%"><rect x="5%" y="10px" width="70%" height="140" fill="rgb('+ str(red_rgb)+","+str(green_rgb)+","+str(blue_rgb)+')" /></svg>'
        display(HTML(html))
        print("R(680nm): ", red )
        print("G(530nm): ", green)
        print("B(465nm): ", blue)
        
    if z == "Make your own":
        for slider in slider_list:
                slider.layout.display = 'flex'
    else:
        with box4:
            clear_output(wait=True)
            classify(RGB,hyperspectral)


        
material = widgets.Dropdown(
    options=["Select",
        "Make your own",
          'Average Amphibian Scat', 
          'Average Bird Scat', 
          'Average Ground',
          "Average Leaf Litter",
          "Average Mammal Scat",
          "Average Reptile Scat",
          "Average Reptile Urea",
          "Average Wood"],
    value='Select',
    description='Material:',
)
output = interactive_output(f, {"z":material,'a':slider1,'b' :slider2, 'c':slider3,'d':slider4,'e' :slider5, 'f':slider6,'g':slider7})
hbox1 = HBox([box1,box2,box3])
hbox2 = HBox([box4,box5])

vbox3 = VBox([material,hbox1,hbox2])
display(vbox3,output)


Output()